In [2]:
import pandas as pd
from plotnine import *

In [40]:
df = pd.read_csv("cache/ashare_daily_2025-09-19.csv")

In [41]:
def clean(df: pd.DataFrame) -> pd.DataFrame:
    df_clean = df.dropna(how="any").copy()  # 删除所有有缺失值的行
    df_clean.iloc[:, 7] = df_clean.iloc[:, 7] / 1e8  # 将数据转换为亿元
    df_clean.iloc[:, 17] = df_clean.iloc[:, 17] / 1e8
    df_clean.iloc[:, 0] = 1  # 将第一行设为1,用于统计个数
    df_clean["净利润"] = df_clean.iloc[:, 17] / df_clean.iloc[:, 15]  # 计算利润
    df_clean["净资产"] = df_clean.iloc[:, 17] / df_clean.iloc[:, 16]  # 计算利润
    df_clean["p_rank"] = df_clean["净利润"].rank(ascending=False)  # 计算利润排名
    df_clean["p_tier"] = (
        df_clean["净利润"].rank(ascending=False, pct=True) * 100
    ).round(3)  # 计算利润百分位
    df_clean["mv_rank"] = df_clean.iloc[:, 17].rank(ascending=False)  # 计算总市值排名
    df_clean["mv_tier"] = (
        df_clean.iloc[:, 17].rank(ascending=False, pct=True) * 100
    ).round(3)  # 计算总市值百分位
    return df_clean


df = clean(df)

In [ ]:
# ''.join([f'({i}:{j})' for i, j in pd.Series(df.columns).items()])
# '(0:序号)(1:代码)(2:名称)(3:最新价)(4:涨跌幅)(5:涨跌额)(6:成交量)(7:成交额)(8:振幅)(9:最高)(10:最低)(11:今开)(12:昨收)(13:量比)(14:换手率)(15:市盈率-动态)(16:市净率)(17:总市值)(18:流通市值)(19:涨速)(20:5分钟涨跌)(21:60日涨跌幅)(22:年初至今涨跌幅)(23:净利润)(24:净资产)(25:p_rank)(26:p_tier)(27:mv_rank)(28:mv_tier)'

'(0:序号)(1:代码)(2:名称)(3:最新价)(4:涨跌幅)(5:涨跌额)(6:成交量)(7:成交额)(8:振幅)(9:最高)(10:最低)(11:今开)(12:昨收)(13:量比)(14:换手率)(15:市盈率-动态)(16:市净率)(17:总市值)(18:流通市值)(19:涨速)(20:5分钟涨跌)(21:60日涨跌幅)(22:年初至今涨跌幅)(23:净利润)(24:净资产)(25:p_rank)(26:p_tier)(27:mv_rank)(28:mv_tier)'

In [42]:
tdic = {
    "停牌数量": df.iloc[:, 7].isna().sum(),
    "停牌占比": df.iloc[:, 7].isna().sum() / len(df),
}

In [43]:
# (0:序号)(1:代码)(2:名称)(3:最新价)(4:涨跌幅)(5:涨跌额)(6:成交量)(7:成交额)(8:振幅)(9:最高)(10:最低)(11:今开)(12:昨收)(13:量比)(14:换手率)(15:市盈率-动态)(16:市净率)(17:总市值)(18:流通市值)(19:涨速)(20:5分钟涨跌)(21:60日涨跌幅)(22:年初至今涨跌幅)(23:profit)(24:p_rank)(25:p_tier)(26:mv_rank)(27:mv_tier)

In [44]:
def plot_points(df: pd.DataFrame, title: str = "title", labn: int = 20):
    cols = df.columns
    lab_df = df.nlargest(labn, cols[1])
    p = (
        ggplot(df, aes(cols[0], cols[1]))
        + geom_point(color="#0394fc")
        + geom_text(aes(label=cols[2]), size=8, data=lab_df)
        + labs(
            title=title,
            x=cols[0],
            y=cols[1],
        )
        + theme_tufte()
        + theme(text=element_text(family="WenQuanYi Micro Hei"))
    )

    return p

In [59]:
sum_s = df.sum()
data = dict(
    股票数=f"{sum_s['序号']}",
    成交额=f"{round(sum_s['成交额'] / 10000, 2)}万亿",
    总市值=f"{round(sum_s['总市值'] / 10000, 2)}万亿",
    净利润=f"{round(sum_s['净利润'] / 10000, 2)}万亿",
    市场PE=f"{round(sum_s['总市值'] / sum_s['净利润'], 2)}",
)
data

{'股票数': '5421',
 '成交额': '2.35万亿',
 '总市值': '117.41万亿',
 '净利润': '5.98万亿',
 '市场PE': '19.62'}

In [55]:
data

{'股票总数': '5421',
 '成交额': '2.35万亿',
 '总市值': '117.41万亿',
 '净利润': '5.98万亿',
 '市场PE': '19.62'}